# Modeling-to-Generate-Alternatives (MGA) Tutorial

MGA is commonly used in energy modelling to address what is known as "structural uncertainty."
That is, the uncertainty stemming from unknown, unmodeled, or unmodel-able objectives. For instance,
political feasibility or some other qualitative variable.

## The MGA Idea

To get around this challenge, MGA searches the "sub-optimal" or "near-optimal" region for alternative
solutions by relaxing the objective function. The goal for a single-objective problem is to find 
"maximally different solutions in the design space." In multi-objective problems, specifically ones solved
with genetic algorithms, users can identify alternatives by random selection or farthest first traversal.

This tutorial will illustrate both methods.

## MGA Example \#1

In [1]:
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def distance_matrix(X, metric='euclidean'):
    """
    This function calculates the distance matrix for an 
    MxN matrix and returns the symmetrical square form of 
    the matrix. 

    Parameters
    ----------
    X : :class:`numpy.ndarray`
        An MxN matrix.
    metric : str
        The string describing how the metric should be calculated.

        See the documentation for 
        [`scipy.spatial.distance.pdist`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html)
        for a complete list of values. Default is 'euclidean.'
    
    Returns
    -------
    D : :class:`numpy.ndarray`
        An MxM matrix of distance values and zeros along the diagonal.
    """

    D = squareform(pdist(X, metric=metric))

    return D

In [6]:
def farthest_first(X, D, n_points, start_idx=None, seed=1234):
    """
    This function identifies the farthest first traversal order for an MxN 
    matrix and returns an array of indices (ordered by the distance). 

    This implementation was modified from Hiroyuki Tanaka's
    [GitHub gist](https://gist.github.com/nkt1546789/8e6c46aa4c3b55f13d32).

    Parameters
    ----------
    X : :class:`numpy.ndarray`
        An MxN matrix.
    D : :class:`numpy.ndarray`
        An MxM distance matrix for the dataset, `X`.
    n_points : int
        The number of points to traverse.
    start_idx : int
        The index of the starting point. If `None`, a starting point
        will be chosen randomly. Default is `None`.
    seed : int
        Specifies the seed for a random number generator to ensure
        repeatable results. Default is 1234. 
    """

    rows, cols = X.shape

    if n_points >= rows:
        return np.arange(rows)
    else:
        checked_points = []

        breakpoint()
        if not start_idx:
            rng = np.random.default_rng(seed)
            start_idx = rng.integers(low=0, high=rows-1)
        
        checked_points.append(start_idx)
        prev_mean_dist = []
        while len(checked_points) < n_points:
            mean_distance = np.mean([D[i] for i in checked_points])
            if mean_distance == prev_mean_dist:
                break
            else:
                sorted_dist = np.argsort(mean_distance)[::-1]
                for j in sorted_dist:
                    if j not in checked_points:
                        checked_points.append(j)
                        break
            prev_mean_dist = mean_distance
    
    return np.array(checked_points)

In [7]:
N = 1  # the number of different technologies or objectives.
pop_size = 5  # the number of unique individuals.
data = np.c_[[np.arange(pop_size) for i in range(pop_size)]]
display(data)
dist = distance_matrix(data)
display(dist)
points = farthest_first(data, dist, n_points=3, start_idx=0)
points

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

/var/folders/6h/g412p7x53jbcqr_x5sy9z8th0000gn/T/ipykernel_47359/1276232678.py:41: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if mean_distance == prev_mean_dist:


array([3, 0])